In [1]:
import sys
sys.path.append('../src/models') 
sys.path.append('../src/data') 
import tensorflow as tf
from tensorflow import keras

In [2]:
from model1.dataloader import load_data
train_dataset, cv_dataset, test_dataset, feature_info = load_data(sample_train= 0.001, feature_info=True)

loading	 train_dataset
loaded:	 train_dataset
loading	 cv_dataset
loaded:	 cv_dataset
loading	 test_dataset
loaded:	 test_dataset
train_dataset: 110 examples
cv_dataset: 12140 examples
test_dataset: 12747 examples


In [3]:
batch_size = 32

train_dataset = train_dataset.batch(batch_size)
train_dataset = train_dataset.shuffle(buffer_size=len(train_dataset))
train_dataset = train_dataset.prefetch(buffer_size=tf.data.AUTOTUNE)

In [4]:
cv_dataset = cv_dataset.batch(batch_size)
test_dataset = test_dataset.batch(batch_size)

In [5]:
feature_info

{'group_feature_size': 463, 'technique_feature_size': 54}

---
# build model from config files

In [6]:
import yaml

In [7]:
with open ('config.yaml', 'r') as config_file:
    config = yaml.safe_load (config_file)

In [8]:
model_architecture_config = config['model_architecture']
traning_config = config['train']

In [9]:
model_architecture_config

{'group_nn_widths': [3, 4, 5, 6],
 'group_nn_depth': 6,
 'technique_nn_widths': [3, 4, 5, 6],
 'technique_nn_depth': 6,
 'nn_output_size': 4}

In [10]:
class customNN(keras.Sequential):
    def __init__(self, 
                 name,
                 input_size, 
                 output_size,
                 widths: int|list,
                 depth: int,
                 hidden_layer_activation = 'relu',
                 output_layer_activation = 'linear',
                 ):
        super().__init__(name = name)
        """
        when there is a single int for `widths`, all the Dense layers are identical in size
        `widths` only indicates the widths in the middle layer, NOT the last layer. 
        The last layer's widths is indicated by `output_size`
        """
        if isinstance (widths, int):
            # First dense layer defined with input shape
            self.add(keras.layers.Dense(widths, input_shape=(input_size,)))
            # Custom layer of hidden Dense layer
            for _ in range (depth-2):
                self.add (keras.layers.Dense(widths,activation= hidden_layer_activation))
            # Output layer
            self.add (keras.layers.Dense (output_size, activation = output_layer_activation, name = 'output_NN'))  
        elif isinstance (widths, list) and len(widths) == (depth-2):
            self.add(keras.layers.Dense(widths[0], input_shape=(input_size,)))
            for width in widths[1:]:
                self.add (keras.layers.Dense(width,activation= hidden_layer_activation))
            self.add (keras.layers.Dense (output_size, activation = output_layer_activation, name = 'output_NN'))  
        else: raise Exception ("CustomNN: widths and depths are set incorrectly.\n Most likely becase: widths is a list, and len(width) == (depth-2) is False")

In [11]:
class customModel(keras.Model):
    def __init__(self, input_sizes,
                 group_nn_widths = None, group_nn_depth = None, 
                 technique_nn_widths = None, technique_nn_depth = None,
                 nn_output_size = None, config = None,
                 *args, **kwargs):
        super().__init__(*args, **kwargs)
        
        if config != None:
            print ('---config FOUND')
            group_nn_widths = config['group_nn_widths']
            group_nn_depth = config['group_nn_depth']
            technique_nn_widths = config['technique_nn_widths']
            technique_nn_depth = config['technique_nn_depth']
            nn_output_size = config['nn_output_size']
            
        group_input_size = input_sizes['group_feature_size']
        technique_input_size = input_sizes['technique_feature_size']
        
        self.input_Group = keras.layers.Input (shape= (group_input_size,), name = 'input_Group')
        self.input_Technique = keras.layers.Input (shape= (technique_input_size,), name = 'input_Technique')
        self.Group_NN = customNN(input_size =  group_input_size,
                                 output_size = nn_output_size,
                                 widths = group_nn_widths,
                                 depth =group_nn_depth,
                                 name = 'Group_NN')
        self.Technique_NN = customNN(input_size = technique_input_size,
                                 output_size = nn_output_size,
                                 widths = technique_nn_widths,
                                 depth = technique_nn_depth,
                                 name = 'Technique_NN')
        
        self.dot_product = keras.layers.Dot(axes= 1)

        
    def call(self, inputs):
        # self.input_NN_1, self.input_NN_2 = inputs[0], inputs[1]
        self.input_Group = inputs['input_Group']
        self.input_Technique = inputs['input_Technique']
        
        output_Group = self.Group_NN(self.input_Group)
        output_Technique = self.Technique_NN(self.input_Technique)
        
        norm_output_Group = tf.linalg.l2_normalize (output_Group, axis = 1)
        norm_output_Technique = tf.linalg.l2_normalize (output_Technique, axis = 1)
        
        dot_product = self.dot_product ([norm_output_Group, norm_output_Technique])
        return dot_product


In [12]:
model = customModel (
    input_sizes= feature_info,
    config= model_architecture_config
)

---config FOUND


---

In [13]:
optimizer = keras.optimizers.Adam (learning_rate= 0.001)    
loss = keras.losses.BinaryCrossentropy (from_logits= True)
model.compile (optimizer, loss = loss)

In [14]:
history = model.fit(
    train_dataset, 
    validation_data= cv_dataset,
    epochs= 1)

4/4 [==============================] - 7s 256ms/step - loss: 0.7267 - val_loss: 0.6944
